In [1]:
import numpy as np
import xobjects as xo
import xtrack as xt
import xpart as xp
import json
import pandas as pd
from cpymad.madx import Madx
#from matplotlib import pyplot as plt
import NAFFlib
from math import modf
#from scipy.constants import physical_constants
from matplotlib import pyplot as plt
from scipy.stats import linregress
import math

In [24]:
print(f"\n\nThis code compares the detuning effect of sextupoles (second order) and octupoles (first order) on N simulated particles\n\n")

#defining some functions to calculate sextupoles and octupoles strength

def oct_str(direction, I_MO, tw, OCT_ON):
    if(OCT_ON == False):
        print("Octupole Strength is zero")
        return 0.0
    with open('hl_line.json', 'r') as fid:
        loaded_dct = json.load(fid)
        line = xt.Line.from_dict(loaded_dct)
    line.vars['i_mo'] = I_MO
    print(f'\n\nCalculating Octupolar strength with i_mo = {I_MO} A')
    sum_nl_x = 0.0
    sum_nl_y = 0.0
    ii = 0
    for elem in line.element_dict :
        if isinstance(line.element_dict[elem],xt.beam_elements.elements.Multipole): 
            if(line.element_dict[elem].order==3):
                    sum_nl_x+=line.element_dict[elem].knl[3]*tw['betx'][ii]*tw['betx'][ii]
                    sum_nl_y+=line.element_dict[elem].knl[3]*tw['bety'][ii]*tw['bety'][ii]
        ii += 1
    m_x = (3*sum_nl_x)/(6*8*np.pi)
    m_y = (3*sum_nl_y)/(6*8*np.pi)
    #print(f'\n\nOctupolar angular Coefficient = {m_x}/m')
    if(direction == 0):
        #print(f'\n\nOctupolar angular Coefficient = {m_x}/m')
        return m_x
    if(direction == 1):
        #print(f'\n\nOctupolar angular Coefficient = {m_y}/m')
        return m_y
    

def sext_strength(direction, tw_normal, sext_name, sext_str, SEXT_ON, doit):
    if(Sextupoles_On == False):
        print("\n\nSextupole Strength is zero")
        return 0.0
    if(doit == False):
        if(direction == 0):
            print(f'\n\nUsing default x Angular Coefficient')
            return 48993.18607814781
        if(direction == 1):
            print(f'\n\nUsing default y Angular Coefficient')
            return 40456.86481785491
    print(f'\n\nCalculating Sextupolar strength')
    nu_x = tw_normal['qx']
    nu_y = tw_normal['qy']
    sext_df = pd.DataFrame(list(zip(sext_name, sext_str)), columns = ['Sextupole Name', 'Sextupole Strength'])
    tw_df = pd.DataFrame({'name':list(tw_normal['name']),'x':list(tw_normal['x']),'px':list(tw_normal['px']),'y':list(tw_normal['y']),
                         'py':list(tw_normal['py']),'zeta':list(tw_normal['zeta']),'delta':list(tw_normal['delta']),
                         'betx':list(tw_normal['betx']),'bety':list(tw_normal['bety']),'mux':list(tw_normal['mux']),'muy':list(tw_normal['muy'])})
    jj = 0
    l_name = []
    l_str = []
    l_betx = []
    l_bety = []
    l_mux = []
    l_muy = []
    for ii in range(len(tw_df['name'])):
        if(tw_df['name'][ii] == sext_df['Sextupole Name'][jj]):
            l_name.append(tw_df['name'][ii])
            l_str.append(sext_df['Sextupole Strength'][jj])
            l_betx.append(tw_df['betx'][ii])
            l_mux.append(tw_df['mux'][ii])
            l_bety.append(tw_df['bety'][ii])
            l_muy.append(tw_df['muy'][ii])
            jj += 1
            if(jj == 1628):
                break
    second_order_df = pd.DataFrame(list(zip(l_name, l_str, l_betx, l_bety, l_mux, l_muy)), 
                                   columns = ['Sextupole Name', 'Sextupole Strength', 'betx', 'bety', 'mux', 'muy'])
    alpha_xx= 0.0
    alpha_yy= 0.0
    for ii in range(len(second_order_df['Sextupole Name'])):
        if(ii%100 == 0):
            print('Iteration number ', ii, 'of', len(second_order_df['Sextupole Name']))
        count = 0
        for jj in range(len(second_order_df['Sextupole Name'])):
            psi_ij_mod_x=np.abs(2*np.pi*second_order_df['mux'][ii]-2*np.pi*second_order_df['mux'][jj])
            psi_ij_mod_y=np.abs(2*np.pi*second_order_df['muy'][ii]-2*np.pi*second_order_df['muy'][jj])

            beta_i_beta_j_x = pow(second_order_df['betx'][ii],3/2)*pow(second_order_df['betx'][jj],3/2)
            beta_i_beta_j_y = pow(second_order_df['betx'][ii],1/2)*pow(second_order_df['betx'][jj],1/2)*second_order_df['bety'][ii]*second_order_df['bety'][jj]

            S_i_S_j = second_order_df['Sextupole Strength'][ii]*second_order_df['Sextupole Strength'][jj]

            I1_x = (np.cos(3*(np.pi*nu_x-psi_ij_mod_x)))/(np.sin(3*np.pi*nu_x))
            I1_y = (np.cos(2*(np.pi*nu_y-psi_ij_mod_y)+(np.pi*nu_x-psi_ij_mod_x)))/(np.sin(np.pi*(2*nu_y+nu_x)))

            I2_x = 3*(np.cos(np.pi*nu_x-psi_ij_mod_x)/(np.sin(np.pi*nu_x)))
            I2_y = (np.cos(2*(np.pi*nu_y-psi_ij_mod_y)-(np.pi*nu_x-psi_ij_mod_x))/(np.sin(np.pi*(2*nu_y-nu_x))))

            I3_y = 4*(np.cos((np.pi*nu_x)-(psi_ij_mod_x)))/(np.sin(np.pi*nu_x))

            alpha_xx += beta_i_beta_j_x*(S_i_S_j)*((I1_x)+(I2_x))
            alpha_yy += beta_i_beta_j_y*(S_i_S_j)*((I1_y)-(I2_y)+(I3_y))

    alpha_xx_tot = -alpha_xx/(64*np.pi)
    alpha_yy_tot = -alpha_yy/(64*np.pi)    
    if(direction == 0):
        #print(f'\n\nSextupolar angular Coefficient, x direction = {alpha_xx_tot}/m')
        return alpha_xx_tot
    if(direction == 1):
        #print(f'\n\nSextupolar angular Coefficient, y direction = {alpha_yy_tot}/m')
        return alpha_yy_tot


    
    














#Parameters needed 
i_mo = -350 #Octupole current, Ampere, zero by default
p0c = 7000e9 #keV
normal_emitt_x = 3e-6 #m*rad
normal_emitt_y = 3e-6 #m*rad
N_particles = 11 
N=1000 #Total number of turns
sampling= 1 #Sampling rate, 1 sample every 'sampling' turns

#The two linear coefficients we want to study
ang_coeff_sext_x = 0.0
ang_coeff_sext_y = 0.0
ang_coeff_oct_x = 0.0
ang_coeff_oct_y = 0.0

#You can decide to turn on one or both
Octupoles_On = True
Sextupoles_On = True
Sextupoles_Sum_x = False
Sextupoles_Sum_y = False
#We need to save sextupole name and strength
sext_name = [] 
sext_str = []


if(Octupoles_On == False):
    i_mo = 0
    print(f"\n\nChanging Octupole Current from -350 A to i_mo = {i_mo} A\n\n")

print(f"\n\nOctupole Current is i_mo = {i_mo} A\n\n")
ctx_cpu = xo.ContextCpu() #Code is intended for CPU

#Opening the line

with open('hl_line.json', 'r') as fid:
        loaded_dct = json.load(fid)
line = xt.Line.from_dict(loaded_dct)

#Reference particle
particle_0 = xp.Particles(mass0=xp.PROTON_MASS_EV, q0=1, p0c=p0c, x=1e-3, y=1e-3)

#Changing the octupole current via knob
line.vars['i_mo'] = i_mo

print(f'\n\nLoading the tracker\n\n')
tracker_normal = xt.Tracker(_context=ctx_cpu, line=line)
ref = tracker_normal.find_closed_orbit(particle_ref = particle_0) 
print(f'\n\nSanity check on closed orbit (CO), the values obtained by the tracker are\n\n')
print(f'\n\nCO.x = {ref.x}, CO.y = {ref.y}, CO.zeta = {ref.zeta} \n\n')

tw_normal = tracker_normal.twiss(ref)

betx_at_ip3 = tw_normal['betx'][0]
bety_at_ip3 = tw_normal['bety'][0]

sigma_x = np.sqrt(betx_at_ip3*normal_emitt_x/(particle_0.gamma0*particle_0.beta0))
sigma_y = np.sqrt(bety_at_ip3*normal_emitt_y/(particle_0.gamma0*particle_0.beta0))

tune_x = tw_normal['qx']
tune_y = tw_normal['qy']

print(f'\n\nBeta values at ip3, betx = {betx_at_ip3} m, bety = {bety_at_ip3} m')
print(f'\n\nSigma values, sigma_x = {sigma_x}, sigma_y = {sigma_y}')

ang_coeff_oct_x = oct_str(0,i_mo, tw_normal, Octupoles_On)
ang_coeff_oct_y = oct_str(1,i_mo, tw_normal, Octupoles_On)
print(f'\n\nThe Octupolar angular Coefficients are m_x= {ang_coeff_oct_x}/m, m_y= {ang_coeff_oct_y}/m')


#----closed orbit particle----------
p0_normal = ref 
#----particles to track----------
particles_x = xp.build_particles(_context=ctx_cpu,particle_ref=p0_normal,
                                     x=[mysigma*sigma_x[0] for mysigma in np.linspace(0.2,2.2,N_particles)])

particles_y = xp.build_particles(_context=ctx_cpu,particle_ref=p0_normal,
                                     y=[mysigma*sigma_y[0] for mysigma in np.linspace(0.2,2.2,N_particles)])


#Saving the starting position
x0 = []
y0 = []
ii = 0
for ii in range(len(particles_x.x)):
    x0.append(particles_x.x[ii])
    y0.append(particles_y.y[ii])

#------Sextupole section-------------
ii = 0
if(Sextupoles_On == False):
    print('\n\nTurning off Sextupoles')
for elem in line.element_dict :
    if isinstance(line.element_dict[elem],xt.beam_elements.elements.Multipole): 
        if((line.element_dict[elem].order==2) and (not Sextupoles_On) ):
            line.element_dict[elem].knl[2]=0
        if(line.element_dict[elem].order==2):
            sext_name.append(elem)
            sext_str.append(line.element_dict[elem].knl[2])
    ii+=1

direction = 0
ang_coeff_sext_x = sext_strength(direction, tw_normal, sext_name, sext_str, Sextupoles_On,Sextupoles_Sum_x)
direction = 1
ang_coeff_sext_y = sext_strength(direction, tw_normal, sext_name, sext_str, Sextupoles_On,Sextupoles_Sum_y)
print(f'\n\nThe Sextupolar angular Coefficients are alpha_xx = {ang_coeff_sext_x}/m, alpha_yy = {ang_coeff_sext_y}')

#-----tracking x direction------------------


my_result_x = {}
for ii in ['x','px','y','py','zeta','delta']:
    my_result_x[ii] = []


for ii in range(N):
    tracker_normal.track(particles_x, num_turns=sampling,turn_by_turn_monitor=False)
    for jj in ['x','px','y','py','zeta','delta']:
        my_result_x[jj].append(getattr(particles_x,jj).copy())       
        
        
for jj in ['x','px','y','py','zeta','delta']:
        my_result_x[jj] = np.array(my_result_x[jj])


        
#-----tracking y direction------------------

my_result_y = {}
for ii in ['x','px','y','py','zeta','delta']:
    my_result_y[ii] = []


for ii in range(N):
    tracker_normal.track(particles_y, num_turns=sampling,turn_by_turn_monitor=False)
    for jj in ['x','px','y','py','zeta','delta']:
        my_result_y[jj].append(getattr(particles_y,jj).copy())       
        
        
for jj in ['x','px','y','py','zeta','delta']:
        my_result_y[jj] = np.array(my_result_y[jj])

qx_part_x = []
qy_part_x = []
qx_part_y = []
qy_part_y = []

for ii in range(N_particles):
    
    qx_part_x.append(NAFFlib.get_tune(my_result_x['x'][:,ii]))
    qy_part_x.append(NAFFlib.get_tune(my_result_x['y'][:,ii]))
    qx_part_y.append(NAFFlib.get_tune(my_result_y['x'][:,ii]))
    qy_part_y.append(NAFFlib.get_tune(my_result_y['y'][:,ii]))
        

jx = np.zeros(N_particles)
tg_phix = -tw_normal['alfx'][0]
phix = np.arctan(tg_phix)
for ii in range(N_particles):
    jx[ii] = x0[ii]*x0[ii]/(2*betx_at_ip3*np.cos(phix)*np.cos(phix))
    

jy = np.zeros(N_particles)
tg_phiy = -tw_normal['alfy'][0]
phiy = np.arctan(tg_phiy)
for ii in range(N_particles):
    jy[ii] = y0[ii]*y0[ii]/(2*bety_at_ip3*np.cos(phiy)*np.cos(phiy))
    
slope_x, intercept, r_value, p_value, std_err = linregress([jx[0],jx[6]],[qx_part_x[0],qx_part_x[6]])
print(f'\n\nThe Simulated Angular Coefficient in the x direction is {slope_x}/m')
print(f'\n\nTheoretical Calculation/Simulation = {(ang_coeff_sext_x+ang_coeff_oct_x)*100/(slope_x)}%')


slope_y, intercept, r_value, p_value, std_err = linregress([jy[0],jy[6]],[qy_part_y[0],qy_part_y[6]])
print(f'\n\nThe Simulated Angular Coefficient in the y direction is {slope_y}/m')
print(f'\n\nTheoretical Calculation/Simulation = {(ang_coeff_sext_y+ang_coeff_oct_y)*100/(slope_y)}%')



This code compares the detuning effect of sextupoles (second order) and octupoles (first order) on N simulated particles




Octupole Current is i_mo = -350 A




Loading the tracker


generating ./928a047fd53245d5831d95aa482cb6f9.c
the current directory is '/afs/cern.ch/user/a/afornara/simulations/sim_stud/git_postprocessing'
running build_ext
building '928a047fd53245d5831d95aa482cb6f9' extension
gcc -pthread -B /home/afornara/py/2022_03_18/miniconda/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/afornara/py/2022_03_18/miniconda/include -I/home/afornara/py/2022_03_18/miniconda/include -fPIC -O2 -isystem /home/afornara/py/2022_03_18/miniconda/include -fPIC -I/home/afornara/py/2022_03_18/miniconda/include/python3.9 -c 928a047fd53245d5831d95aa482cb6f9.c -o ./928a047fd53245d5831d95aa482cb6f9.o -std=c99 -O3 -Wno-unused-function
gcc -pthread -B /home/afornara/py/2022_03_18/miniconda/compiler_compat -shared -Wl,-rpath,/home/afornara/py/2022_03